In [1]:

import os
import keras
import json
import glob
import sys
import pandas as pd
import tensorflow as tf
from keras.layers import Input
import numpy as np
import argparse
#from keras_applications.resnext import ResNeXt50
from keras.utils.data_utils import get_file
#import face_recognition
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add,Normalization
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
! pip install keras_tuner
import keras_tuner as kt
from keras import backend as K
#sys.path.append('/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML')
import fnmatch
import re
import copy
import matplotlib.pyplot as plt
import scipy.signal
! pip install mpu
import mpu
#from script_onelist_colab import norm_int,repair,csv_exp,sel_exp,maxima,import_raw,read_processed,read_processed_sel,correct,onelist,interpolate_mat,interpolation_spectra

def touch(fname):
    if os.path.exists(fname):
        os.utime(fname, None)
    else:
        os.mkdir(fname)

#folder='/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML/csv/'
#if not os.path.abspath('')=='/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML/csv':
#  os.chdir(folder)
#print(os.path.abspath(''))  
#touch(folder+'png')





In [2]:
datadict={}

In [3]:
def noise(source):
  noisy=copy.deepcopy(source)
  smax=np.amax(noisy[:,1:])
  for j in range(len(noisy[0,:])):
    for i in range(len(noisy[:,0])):
      if j>0:
        noisy[i,j]+=np.random.uniform(-2,2)
        noisy[i,j]+=np.random.uniform(-1,1)*0.01*noisy[i,j]
  print(np.amax(noisy[:,1:]))
  noisy[:,1:]=noisy[:,1:]/np.amax(noisy[:,1:])*smax
  noisy[:,1:][noisy[:,1:] < 0] = 0
  noisy[:,1:]=np.rint(noisy[:,1:])
  #print(np.shape(noisy))
  return noisy


In [4]:
files=glob.glob("csv/png/sel_obs2nm_blank_3000uleth*.csv")

In [5]:
    fileslist=dict.fromkeys(files,1)
    i=1
    for key in fileslist:
        td=pd.read_csv(key,index_col=0)
        # col[key[4:-4]+'.txt']=td.columns
        tdn=np.array(td)
        datadict[key[8:-4]+'.txt']=tdn.astype(np.float)
        i+=1

In [6]:
#df3_copy=copy.deepcopy(df3)

def add_blank(x):
  if 'blank' not in x.file_name:
    ret=df2.loc[(df2['CD_amount'] == 'blank') & (df2['typ'] == x['typ']),'spec'].iloc[0]
    print(ret)
  else:
    ret=np.nan
  return noise(ret)

In [7]:
def fill(x,source):
  y=x
  for col in source.columns:

    if np.isnan(np.amax(np.amax(x[col]))):
      nancheck=0
      while nancheck==0:
        newval=source[col].iloc[np.random.randint(0,len(source))]

        if not np.isnan(np.amax(np.amax(newval))):
          print('yes')
          nancheck=1
     # print(np.shape(noise(newval)))
      y[col]=noise(newval)    
  return y

In [8]:
df3=pd.read_pickle('csv/PL_LT_sol_aug_pd.pkl')

In [9]:
checker3='2nm_CD032_0.5_(.*)ul_(.*)ulethanol_(.*)ulwater_(.*)_(.*).txt'
checker='sel_obs2nm_blank_3000uleth(.*)_(.*)_(.*)'
df2=pd.DataFrame(index=None,columns=('alcohol_concentration','CD_amount','typ','file_name','spec'))
for key in datadict: 
  if re.search(checker,key) is not None: 
    con='blank'
    am='blank'
    typ=re.search(checker,key).group(2)
    dat=pd.DataFrame([[con,am,typ,key,datadict[key]]]) 
    dat.columns=df3.columns 
    #df3=df3.append(dat,ignore_index=True) 
    df2=df2.append(dat,ignore_index=True) 

In [10]:
df3=df3.drop(df3.loc[df3['typ']=='LT'].index)

In [11]:
df3['blank_spec']=np.nan
for typ in df3['typ'].unique():
  print(typ)
  df3['blank_spec'] = df3.apply(lambda x: add_blank(x) if x['typ']==str(typ) else x['blank_spec'], axis=1)


PL
[[ 300. 2058. 2011. ...    0.    0.    0.]
 [ 320.    0.    0. ...    0.    0.    0.]
 [ 340.    0.    0. ...    0.    0.    0.]
 ...
 [ 560.    0.    0. ...   71.    0.    0.]
 [ 580.    0.    0. ...   62.   43.   62.]
 [ 600.    0.    0. ...   33.  107.   70.]]
3025.505324255856
[[ 300. 2058. 2011. ...    0.    0.    0.]
 [ 320.    0.    0. ...    0.    0.    0.]
 [ 340.    0.    0. ...    0.    0.    0.]
 ...
 [ 560.    0.    0. ...   71.    0.    0.]
 [ 580.    0.    0. ...   62.   43.   62.]
 [ 600.    0.    0. ...   33.  107.   70.]]
2994.373551666832
[[ 300. 2058. 2011. ...    0.    0.    0.]
 [ 320.    0.    0. ...    0.    0.    0.]
 [ 340.    0.    0. ...    0.    0.    0.]
 ...
 [ 560.    0.    0. ...   71.    0.    0.]
 [ 580.    0.    0. ...   62.   43.   62.]
 [ 600.    0.    0. ...   33.  107.   70.]]
3023.501701195947
[[ 300. 2058. 2011. ...    0.    0.    0.]
 [ 320.    0.    0. ...    0.    0.    0.]
 [ 340.    0.    0. ...    0.    0.    0.]
 ...
 [ 560.    0.    

In [12]:
df3=df3.drop(df3.loc[df3['CD_amount']=='blank'].index)

In [13]:
df3['blank_LT405spec'] = df3.apply(lambda x: x['blank_spec'] if x['typ']=='LT405' else np.nan, axis=1)
df3['blank_LT450spec'] = df3.apply(lambda x: x['blank_spec'] if x['typ']=='LT450' else np.nan, axis=1)
df3['blank_PLspec'] = df3.apply(lambda x: x['blank_spec'] if x['typ']=='PL' else np.nan, axis=1)

In [14]:
df3['LT405spec'] = df3.apply(lambda x: x['spec'] if x['typ']=='LT405' else np.nan, axis=1)
df3['LT450spec'] = df3.apply(lambda x: x['spec'] if x['typ']=='LT450' else np.nan, axis=1)
df3['PLspec'] = df3.apply(lambda x: x['spec'] if x['typ']=='PL' else np.nan, axis=1)


In [15]:
df3=df3.drop('blank_spec',axis=1)

In [16]:
for amount in df3['CD_amount'].unique():
  print(amount)
  df3[str(int(amount))+'_LT405spec'] = df3.apply(lambda x: x['LT405spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  df3[str(int(amount))+'_LT450spec'] = df3.apply(lambda x: x['LT450spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  df3[str(int(amount))+'_PLspec'] = df3.apply(lambda x: x['PLspec'] if x['CD_amount']==int(amount) else np.nan, axis=1)

300.0
100.0
150.0


In [17]:
df3=df3.drop('LT405spec',axis=1)
df3=df3.drop('LT450spec',axis=1)
df3=df3.drop('PLspec',axis=1)

In [18]:
df3=df3.drop('spec',axis=1)

In [19]:
cl=df3.iloc[:,4:].columns
c=df3.iloc[:,7:].columns

In [20]:
df4=pd.DataFrame(columns=df3.columns)

In [21]:

for alccon in df3['alcohol_concentration'].unique():
  print(alccon)
  df_tmp=copy.deepcopy(df3.loc[df3['alcohol_concentration']==alccon])
  df_tmp= df_tmp.apply(lambda x: pd.Series(x.dropna().values))
  no=max(df_tmp[c].count())
  df_tmp=df_tmp.iloc[:no,:]
  df_tmp=df_tmp.apply(lambda x: fill(x,df_tmp[cl]) ,axis=1)
  df4=pd.concat((df4,df_tmp))
  del df_tmp
  #df3[str(int(amount))+'_LT405spec'] = df3.apply(lambda x: x['LT405spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  #df3[str(int(amount))+'_LT450spec'] = df3.apply(lambda x: x['LT450spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  #df3[str(int(amount))+'_PLspec'] = df3.apply(lambda x: x['PLspec'] if x['CD_amount']==int(amount) else np.nan, axis=1)

0.8
yes
65422
yes
64719
yes
65535
yes
65259
yes
64934
yes
65527
yes
64953
yes
65320
yes
65535
yes
64578
yes
65459
yes
64680
yes
65533
yes
65221
yes
64324
yes
65531
yes
65284
yes
65062
yes
65198
yes
65532
yes
65204
yes
64912
yes
65534
yes
64190
yes
65249
yes
65457
yes
65532
yes
64147
yes
64268
yes
65532
yes
65296
yes
65386
yes
65334
yes
65528
1.0
yes
65151
yes
64439
yes
65524
yes
64464
yes
65530
yes
65494
yes
65456
yes
65533
0.4666666666666667
yes
64779
yes
65468
yes
64776
yes
64869
yes
65509
0.06666666666666667
yes
65106
yes
65099
yes
65535
yes
62967
yes
65166
yes
65534
yes
65145
yes
64537
yes
65528
yes
65433
yes
65427
yes
65405
yes
62269
yes
65534
0.6333333333333333
yes
64557
yes
64933
yes
65531
yes
64448
yes
64810
yes
65535
0.9666666666666667
yes
65505
yes
65530
yes
65434
yes
65535
yes
65263
yes
65384
yes
65535
yes
64928
yes
64282
yes
65535
yes
65222
yes
65310
yes
64647
yes
65346
yes
65534
yes
65163
yes
65166
yes
65535
yes
64692
yes
65094
yes
65238
yes
65306
yes
65534
0.2
yes
65232
y

In [22]:
df4=df4.reset_index(drop=True)
df4=df4.drop(['CD_amount','typ','file_name'],axis=1)

In [23]:
df4.to_pickle('PL_LT_augmented_onemat_pd2.pkl')

In [24]:
df4=pd.read_pickle('PL_LT_augmented_onemat_pd2.pkl')

In [25]:
df4

,alcohol_concentration,blank_LT405spec,blank_LT450spec,blank_PLspec,300_LT405spec,300_LT450spec,300_PLspec,100_LT405spec,100_LT450spec,100_PLspec,150_LT405spec,150_LT450spec,150_PLspec
0,0.800000,"[[0.0, 63.0], [1.0, 65.0], [2.0, 55.0], [3.0, ...","[[0.0, 32.0], [1.0, 31.0], [2.0, 42.0], [3.0, ...","[[300.0, 2024.0, 2011.0, 2111.0, 2289.0, 2626....","[[0, 58], [1, 58], [2, 63], [3, 68], [4, 62], ...","[[0, 55], [1, 55], [2, 61], [3, 74], [4, 66], ...","[[300, 0, 0, 7, 0, 3, 0, 0, 4, 1, 0, 11, 0, 6,...","[[0, 74], [1, 74], [2, 65], [3, 57], [4, 58], ...","[[0, 63], [1, 63], [2, 68], [3, 74], [4, 64], ...","[[300, 0, 4, 6, 5, 0, 14, 0, 10, 2, 16, 2, 0, ...","[[0, 64], [1, 64], [2, 64], [3, 53], [4, 54], ...","[[0, 62], [1, 62], [2, 43], [3, 69], [4, 66], ...","[[300, 4, 0, 0, 14, 7, 7, 13, 10, 3, 8, 12, 16..."
1,0.800000,"[[0.0, 63.0], [1.0, 64.0], [2.0, 59.0], [3.0, ...","[[0.0, 30.0], [1.0, 31.0], [2.0, 44.0], [3.0, ...","[[300.0, 2040.0, 1995.0, 2123.0, 2286.0, 2606....","[[0, 43], [1, 84], [2, 36], [3, 109], [4, 59],...","[[0, 24], [1, 55], [2, 29], [3, 90], [4, 117],...","[[300, 0, 0, 0, 1, 2, 10, 6, 3, 0, 8, 4, 4, 0,...","[[0, 99], [1, 99], [2, 111], [3, 99], [4, 98],...","[[0, 88], [1, 88], [2, 99], [3, 99], [4, 92], ...","[[300, 1, 0, 0, 4, 9, 0, 5, 20, 21, 14, 24, 23...","[[0, 88], [1, 23], [2, 70], [3, 90], [4, 59], ...","[[0, 35], [1, 13], [2, 13], [3, 80], [4, 55], ...","[[300, 8, 0, 0, 2, 0, 1, 5, 5, 1, 1, 1, 4, 0, ..."
2,0.800000,"[[0.0, 62.0], [1.0, 62.0], [2.0, 56.0], [3.0, ...","[[0.0, 29.0], [1.0, 32.0], [2.0, 42.0], [3.0, ...","[[300.0, 2034.0, 2015.0, 2119.0, 2281.0, 2605....","[[0, 42], [1, 82], [2, 35], [3, 108], [4, 57],...","[[0, 23], [1, 55], [2, 27], [3, 90], [4, 116],...","[[300, 0, 0, 12342, 0, 0, 1, 0, 0, 0, 1, 0, 0,...","[[0, 61], [1, 75], [2, 70], [3, 104], [4, 10],...","[[0, 52], [1, 52], [2, 58], [3, 63], [4, 47], ...","[[300, 2, 0, 0, 7, 15, 6, 18, 14, 9, 11, 20, 1...","[[0, 86], [1, 21], [2, 70], [3, 91], [4, 58], ...","[[0, 63], [1, 61], [2, 44], [3, 67], [4, 64], ...","[[300, 3, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, ..."
3,0.800000,"[[0.0, 63.0], [1.0, 62.0], [2.0, 59.0], [3.0, ...","[[0.0, 30.0], [1.0, 30.0], [2.0, 41.0], [3.0, ...","[[300.0, 2045.0, 2005.0, 2094.0, 2254.0, 2611....","[[0, 43], [1, 82], [2, 34], [3, 110], [4, 59],...","[[0, 54], [1, 52], [2, 60], [3, 72], [4, 64], ...","[[300, 0, 12380, 12408, 0, 0, 1, 0, 0, 0, 0, 0...","[[0, 74], [1, 73], [2, 62], [3, 57], [4, 58], ...","[[0, 104], [1, 26], [2, 57], [3, 102], [4, 85]...","[[300, 0, 0, 0, 0, 4, 0, 0, 2, 10, 6, 4, 7, 18...","[[0, 87], [1, 21], [2, 68], [3, 88], [4, 56], ...","[[0, 61], [1, 60], [2, 41], [3, 69], [4, 64], ...","[[300, 3, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
4,0.800000,"[[0.0, 65.0], [1.0, 66.0], [2.0, 59.0], [3.0, ...","[[0.0, 31.0], [1.0, 31.0], [2.0, 44.0], [3.0, ...","[[300.0, 2044.0, 2014.0, 2114.0, 2290.0, 2619....","[[0, 59], [1, 57], [2, 63], [3, 67], [4, 62], ...","[[0, 55], [1, 56], [2, 61], [3, 72], [4, 68], ...","[[300, 0, 0, 0, 0, 0, 1, 1, 0, 12360, 1, 0, 0,...","[[0, 99], [1, 100], [2, 107], [3, 98], [4, 99]...","[[0, 22], [1, 47], [2, 75], [3, 75], [4, 67], ...","[[300, 5, 0, 0, 4, 4, 0, 8, 0, 6, 4, 4, 2, 15,...","[[0, 63], [1, 64], [2, 63], [3, 52], [4, 54], ...","[[0, 63], [1, 61], [2, 43], [3, 69], [4, 66], ...","[[300, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,0.320000,"[[0.0, 63.0], [1.0, 64.0], [2.0, 58.0], [3.0, ...","[[0.0, 33.0], [1.0, 30.0], [2.0, 42.0], [3.0, ...","[[300.0, 2040.0, 1990.0, 2134.0, 2285.0, 2635....","[[0, 76], [1, 49], [2, 68], [3, 48], [4, 54], ...","[[0, 91], [1, 39], [2, 39], [3, 112], [4, 0], ...","[[300, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 43, 0, 0,...","[[0, 128], [1, 108], [2, 94], [3, 107], [4, 90...","[[0, 68], [1, 121], [2, 121], [3, 96], [4, 81]...","[[300, 8, 4, 0, 11, 2, 9, 6, 12, 3, 5, 6, 0, 8...","[[0, 64], [1, 42], [2, 0], [3, 99], [4, 50], [...","[[0, 107], [1, 33], [2, 73], [3, 29], [4, 31],..."